In this lab, we shall focus on language modeling using n-grams.


Following is the code to generate n-grams with their frequencies from a text document.

In [3]:
import nltk
from nltk.util import ngrams
from collections import Counter
text=u"Hello how are you doing? Hope you find the course interesting. How are you doing?"
tokens=text.split() #basic tokenization

## Printing vocabulary size by creating a set from tokens
print ("Words:: ",tokens)
vocabulary=set(tokens)
print("\nVocabulary size:: ",len(vocabulary))

#generating bi-grams from tokens
bitokens=ngrams(tokens,2)

# using a counter function to count the bigrams
ngramsWithFreq=Counter(bitokens)
print ("\nTop 5::  ", ngramsWithFreq.most_common(5))



Words::  ['Hello', 'how', 'are', 'you', 'doing?', 'Hope', 'you', 'find', 'the', 'course', 'interesting.', 'How', 'are', 'you', 'doing?']

Vocabulary size::  11

Top 5::   [(('are', 'you'), 2), (('you', 'doing?'), 2), (('Hello', 'how'), 1), (('how', 'are'), 1), (('doing?', 'Hope'), 1)]


Following is an alternate mechanism to generate n-grams. We are using collocation finder class in the following code. Collocation finder helps use in finding collocated words--words appearing together.

In the following code, ngram_fd is an attribute of the instance "trigrams" that keeps the frequency distribution of all the n-grams. ngram_fd.items() is a function that returns the list of all n-grams with their frequencies. To find about different functions and attributes in the instance "trigrams", type "dir(trigrams)" and to get the description of the listed functions use help; e.g., "help(trigrams.ngram_fd)".

In [7]:
from nltk.collocations import *
import nltk
text=u"Hello how are you doing? Hope you find the book interesting."

#tokenize
tokens=nltk.word_tokenize(text)

#Using a trigram collocation finder
trigrams=TrigramCollocationFinder.from_words(tokens)

# printing trigrams
for trigram, freq in trigrams.ngram_fd.items():
    print(trigram,freq)

('Hello', 'how', 'are') 1
('how', 'are', 'you') 1
('are', 'you', 'doing') 1
('you', 'doing', '?') 1
('doing', '?', 'Hope') 1
('?', 'Hope', 'you') 1
('Hope', 'you', 'find') 1
('you', 'find', 'the') 1
('find', 'the', 'book') 1
('the', 'book', 'interesting') 1
('book', 'interesting', '.') 1


In the following code, we are continuing with the previous example and we are going to filter out stop words and print top few n-grams. In order to get top n-grams, we will use nbest funtion. The nbest function can return n-grams based on a scoring criterion. In this case, we are scoring by frequency, so we will use TrigramAssocMeasures class and its attribute raw_freq. There are other methods of scoring too in the TrigramAssocMeasures class, which you can find out by using help(TrigramAssocMeasures)

In [8]:
from nltk.collocations import *
from nltk.corpus import stopwords
from nltk.metrics import TrigramAssocMeasures
from nltk.tokenize import TreebankWordTokenizer

text=u"Do you know a good mentor may help? Hope you find the book interesting but a good mentor may help. Definitely the book is interesting and good"

# Tokenzing using treebank tokenizer
tokenizer = TreebankWordTokenizer()
tokens=tokenizer.tokenize(text)

stopWords = set(stopwords.words('english'))


filteredTokens=[t.lower() for t in tokens if t.lower() not in stopWords]

trigrams=TrigramCollocationFinder.from_words(filteredTokens)

for trigram, freq in trigrams.ngram_fd.items():
   print(trigram,freq)

print (":::Top n-grams:::")
trigrams.nbest(TrigramAssocMeasures.raw_freq, 5)


('know', 'good', 'mentor') 1
('good', 'mentor', 'may') 2
('mentor', 'may', 'help') 1
('may', 'help', '?') 1
('help', '?', 'hope') 1
('?', 'hope', 'find') 1
('hope', 'find', 'book') 1
('find', 'book', 'interesting') 1
('book', 'interesting', 'good') 2
('interesting', 'good', 'mentor') 1
('mentor', 'may', 'help.') 1
('may', 'help.', 'definitely') 1
('help.', 'definitely', 'book') 1
('definitely', 'book', 'interesting') 1
:::Top n-grams:::


[('book', 'interesting', 'good'),
 ('good', 'mentor', 'may'),
 ('?', 'hope', 'find'),
 ('definitely', 'book', 'interesting'),
 ('find', 'book', 'interesting')]

Time to measure some probabilities. Following code shows probability measurements using  basic MLE (Maximum Likelihood Estimate) and Laplace smoothing

In [ ]:
import nltk
from nltk.probability import *
from nltk.tokenize import word_tokenize

text = 'This is an example sentence, example sentence, example sentence example sentence, example sentence'
tokens=word_tokenize(text)

# First get the frequency distribution of tokens by FreqDist class
fdist=FreqDist(token.lower() for token in tokens)
# Verify the frequency distribution is correct
print ("Frequency distribution")
print(fdist.items())

#Measuring  probabilities
print ("\nProbability measurements")
mle=MLEProbDist(fdist)
print("MLE for 'example': ",mle.prob("example"))
print("MLE for 'this': ",mle.prob("this"))
print("MLE for 'unknown': ",mle.prob("unkown"))

lpl=LaplaceProbDist(fdist)
print("\nLaplace for 'example': ",lpl.prob("example"))
print("Laplace for 'this': ",lpl.prob("this"))
print("Laplace for 'unknown': ",lpl.prob("unkown"))


Below, we shall see some examples of trigrams based smoothing using Laplace and Kneser Ney algorthms. 

In [ ]:
# We have all sentences separately stored in lists to get correct n-grams
text = ['I am Sam', ' Sam I am', 'I do not like green eggs and ham']
trigramTokens = []

for sent in text:
    words=word_tokenize(sent)
    trigramTokens+=nltk.trigrams(words,pad_left=True, pad_right=True, 
                    left_pad_symbol='<S>', right_pad_symbol='</S>')

### Try to print all of above tri-grams yourself to have an idea of what they look like and
### try removing parameters with pad keyword or see the help to understand what is the purpose
### of them

 
## Geting an instance of fequency distribution    
fdisTri= FreqDist(trigramTokens)

## Initializing Kneser Ney Probabilities by using trigrams
## Kneser Ney does not work without trigrams
knsr=KneserNeyProbDist(fdisTri)
print ("\n::: Kneser Ney :::")
for tup in knsr.samples():
    print (tup," = ", knsr.prob(tup))
   

#Now time for Laplace on trigrams
print ("\n:::: Laplace :::")
lpl=LaplaceProbDist(fdisTri)
for tup in lpl.samples():
    print (tup," = ", lpl.prob(tup))
    





If we dissect above code, we shall realize that we have created joint probability distributions for N-grams above using different probability measurement methods. For example, see below. The frequency of (<S,<S, I) is 2 (c), total counts of all trigrams is 20 (N) and unique trigrams is 19 (B). The Laplace prob is calculated as: c+1/(N+B) = (2+1)/(20+19)=0.07.
This simply gives the joint probabilities (intersection probs) and we need to calculate conditional probability distribution for trigrams by replacing N with the counts of bigrams. 

In [ ]:
print(fdisTri.items())
print( "Total Count", fdisTri.N())
print ("Distinct Count", fdisTri.B())

So here is an example of conditional probability distribution using Laplace smoothing. First we calculate conditional frequency distributions and then we calulate conditional probabilities based on Laplace smoothing. Go through the code and comments to understand it further. In this code, we are also measuring the perplexity of the model by using the equations: 

Prod= (Product-for-all-trigrams-in-test(P (w3|w2,w1) ) and Power(Prod, -1/(tokens in test))

In [9]:
import math
from nltk.probability import ConditionalFreqDist
from nltk.tokenize import word_tokenize
from nltk.probability import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, LaplaceProbDist
sent= "Here is some repetitive and here is some repetitive text and is some"


# get some tokens
tokens=[word.lower() for word in word_tokenize(sent)]
# get the size of vocabulary
vocabSize=set(tokens).__len__()
print ("Vocabulary size:",vocabSize)

# Get trigrams
tri = nltk.trigrams(tokens,pad_left=True, pad_right=True, 
                    left_pad_symbol='<S>', right_pad_symbol='</S>' ) 

# Measuring conditional frequency distributions
cfDist = ConditionalFreqDist(((w1,w2),w3) for w1,w2,w3 in tri)

#Above line of code is equivalent to the following three lines of code
#for w1,w2,w3 in trigrams:
#   condition = (w1,w2)
#   cfdist[condition][w3] += 1

# Let's verify conditional frequencies for some random w1,w2
fDist=cfDist[('here','is')]     
print ("Items following 'here is':", fDist.items())

fDist=cfDist[('is','some')]     
print ("Items following 'is some':", fDist.items())


cpDist = ConditionalProbDist(cfDist, nltk.probability.LaplaceProbDist, bins=vocabSize)



print("P(repetitive|is,some)=",cpDist[('is','some')].prob('repetitive'))
print("P(some|is,some)=",cpDist[('is','some')].prob('more'))
print("P(is|is,some)=",cpDist[('is','some')].prob('is'))
print("P(the|the,the)=",cpDist[('the','the')].prob('the'))


print ("\n:::: Now calcualte perplexity on a test sentence :::")

test = 'here is some'
tri=nltk.trigrams(word_tokenize(test),
                  pad_left=True, pad_right=True, 
                  left_pad_symbol='<S>', right_pad_symbol='</S>' ) 
probTest=1.0 
tokenCount=0;
for w1,w2,w3 in tri:
    pr=cpDist[(w1,w2)].prob(w3)
    print ("P(",w3,"| ",w1,w2,")=", pr )
    probTest=probTest * pr
    tokenCount=tokenCount+1


print ("Probability of the test text: ",probTest)
perplexity=math.pow(probTest,-(1/tokenCount))
print("Perplexity:", math.pow(probTest,-(1/tokenCount)))




Vocabulary size: 6
Items following 'here is': dict_items([('some', 2)])
Items following 'is some': dict_items([('repetitive', 2), ('</S>', 1)])
P(repetitive|is,some)= 0.3333333333333333
P(some|is,some)= 0.1111111111111111
P(is|is,some)= 0.1111111111111111
P(the|the,the)= 0.16666666666666666

:::: Now calcualte perplexity on a test sentence :::
P( here |  <S> <S> )= 0.2857142857142857
P( is |  <S> here )= 0.2857142857142857
P( some |  here is )= 0.375
P( </S> |  is some )= 0.2222222222222222
P( </S> |  some </S> )= 0.2857142857142857
Probability of the test text:  0.001943634596695821
Perplexity: 3.485596218940725


We will repeat the same process as above with Kneser Ney smoothing. Kneser Ney smoothing requires all the words in trigram to be passed to it for each condition (see below). The perplexity of the model on test set with Kneser Ney smoothing is lower than Laplace smoothing. However, there is a problem with the implementation of Kneser Ney in the code of NLTK. It is returning 0 values for any trigram pattern even if the word is already present in the vocabulary which does not seem to satisfy smoothing concept. Also, if you slightly change the training data, it would produce 0 probability and won't be comprable to Laplace smoothing above. Let me know if you find out that NLTK implementation is correct and there is something wrong in the code.

In [ ]:
sent= "Here is some repetitive and here is some repetitive text and is some"


# get some tokens
tokens=[word.lower() for word in word_tokenize(sent)]
# get the size of vocabulary
vocabSize=set(tokens).__len__()
print ("Vocabulary size:",vocabSize)

# Get trigrams
tri = nltk.trigrams(tokens,pad_left=True, pad_right=True, 
                  left_pad_symbol='<S>', right_pad_symbol='</S>')

cfdist2 = ConditionalFreqDist(((w1,w2),(w1,w2,w3)) for w1,w2,w3 in tri)

fDist2=cfdist2[('here','is')]     
print ("Items following 'here is':", fDist2.items())
cpdist2 = ConditionalProbDist(cfdist2, nltk.probability.KneserNeyProbDist, bins=vocabSize)

print(cpdist2[('is','some')].prob(['is','some','repetitive']))
print(cpdist2[('is','some')].prob(['is','some','and']))
print(cpdist2[('is','some')].prob(['is','some','the']))



print ("\n:::: Now calcualte perplexity on a test sentence :::")

test = 'here is some'
tri=nltk.trigrams(word_tokenize(test),
                  pad_left=True, pad_right=True, 
                  left_pad_symbol='<S>', right_pad_symbol='</S>' ) 
probTest=1.0 
tokenCount=0;
for w1,w2,w3 in tri:
    pr=cpdist2[(w1,w2)].prob([w1,w2,w3])
    print ("P(",w3,"| ",w1,w2,")=", pr )
    probTest=probTest * pr
    tokenCount=tokenCount+1


print ("Probability of the test text: ",probTest)
perplexity=math.pow(probTest,-(1/tokenCount))
print("Perplexity:", math.pow(probTest,-(1/tokenCount)))




Here are some exercises for you:

2.2) Implement Laplace smoothing for bigrams and unigrams. Measure the preplexity on the test set and comapre it the result of trigram application.

2.3) Implement another equation of perplexity which yileds the same result as the equation used above. Here is the equation in three steps: (a) Logbase2Prob= Sum-for-all-trigrams(log2(P(w3|w1,w2)) (b)Ent=(-1/tokens-in-test)x Logbase2Prob (c) Power(2, (ent) )

2.4) Wrtie a program to predict  next words given that you already know some words. THis kind of application, you must have already seen on mobile phone keyboards, in Word processing softwares ot elsewhere. This is very easy to implement, you will need to perform training of your data on some text (as above) and in testing you will predict the next word of a given sequence of words by using trigrams (or bigrams). The initial part of the bigram/trigram will be first few words in your test sequence of words and the last part will be the one that has the maximum probability given the previous parts. If you see the help on the probability distribution on NLTK, there are already functions available to solve this.
